# Creating Fact and Dimension Tables with Star Schema in PostgresSQL

In [1]:
import psycopg2

### Let's create the connection to the db and get a cursor for query.

In [2]:
try:
    conn = psycopg2.connect("host=localhost user=postgres password=uncle1dee")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)

conn.set_session(autocommit=True)    


### Create the database

In [3]:

try:
    cur.execute("create database udacity")
except psycopg2.Error as e:
    print(e)

database "udacity" already exists



### Let create the Fact Table and Insert the data into the table

In [4]:
# Create all Table

try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer_transactions (customer_id int, store_id int, spent numeric);") # Fact table
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
    
# Insert into the tables
    
try:
    cur.execute("INSERT INTO customer_transactions (customer_id, store_id, spent) VALUES (%s, %s, %s)",
                (1, 1, 20.50))
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO customer_transactions (customer_id, store_id, spent) VALUES (%s, %s, %s)",
                (2, 1, 35.21))
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)

### Let's create the Dimension Table and insert the data

In [5]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS items_purchased (customer_id int, item_number int, item_name varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

# Insert into the tables
try:
    cur.execute("INSERT INTO items_purchased (customer_id, item_number, item_name) VALUES (%s, %s, %s)",
                (1, 1, "Rubber Soul"))
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO items_purchased (customer_id, item_number, item_name) VALUES (%s, %s, %s)",
                (2, 3, "Let It Be"))
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)

        
try:
    cur.execute("CREATE TABLE IF NOT EXISTS store (store_id int, state varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
    
# Insert into the tables

try:
    cur.execute("INSERT INTO store (store_id, state) VALUES (%s, %s)",
                (1, "CA"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)


try:
    cur.execute("INSERT INTO store (store_id, state) VALUES (%s, %s)",
                (2, "WA"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
        
try:
    cur.execute("CREATE TABLE IF NOT EXISTS customer (customer_id int, name varchar, reward boolean);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
 
 
# Insert into the tables
    

try:
    cur.execute("INSERT INTO customer (customer_id, name, reward) VALUES (%s, %s, %s)",
                (1, "Amanda", True))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        

try:
    cur.execute("INSERT INTO customer (customer_id, name, reward) VALUES (%s, %s, %s)",
                (2, "Toby", False))
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)             


### We can do a variety of queries on his data easily because of utilizing the fact/dimension and Star schema
- Query 1: Find all the customers that spent more than 30 dollars, who are they, what did they buy and if they are a rewards member
- Query 2: How much did store 1 sell

##### Query 1: Find all the customers that spent more than 30 dollars, who are they, what did they buy and if they are a rewards member

In [6]:
try:
    cur.execute("SELECT name, item_name, reward FROM ((customer_transactions \
    JOIN customer on customer.customer_id=customer_transactions.customer_id)\
    JOIN items_purchased ON \
    customer_transactions.customer_id=items_purchased.customer_id)\
    WHERE spent > 30;")
    
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Toby', 'Let It Be', False)


#### - Query 2: How much did store 1 sell

In [7]:
try:
    cur.execute("SELECT store_id, sum(spent) from customer_transactions group by store_id;")
except psycopg2.Error as e:
    print("Error: Select*")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, Decimal('55.71'))


### Summary: What you can see here is from this elegant schema we were able to get "Facts/Metrics from our fact table (how much each store sold), and also information about our customers that will allow us to do more indepth analytics to get answers to business questions by utilizimg our fact and dimension tables

### Drop all the tables

In [8]:
    
try:
    cur.execute("DROP table customer_transactions")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

try:
    cur.execute("DROP table items_purchased")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

try:
    cur.execute("DROP table customer")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)
    
try:
    cur.execute("DROP table store")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)


### And Finally close your cursor and connection.

In [9]:
cur.close()
conn.close()